In [1]:
import os
import sys
import time
from llama_cpp import Llama

In [ ]:
MODEL_DIR = "C:/Users/bsjun/.lmstudio/models/lmstudio-community/"
MODEL_NAME = "EXAONE-3.5-2.4B-Instruct-GGUF"

DEVICE = "cpu"

SYSTEM_PROMPT = """당신은 카드사 고객센터에 문의하는 '고객'입니다.
절대로 정보를 제공하는 역할을 하지 말고, 사용자의 물음에 계속해서 질문을 하는 고객임을 명심하세요.

[지시사항]
1. 상담원(사용자)에게 카드 이용, 한도, 결제 등과 관련된 질문을 계속 던지세요.
2. 사용자의 답변이 이해가 안 가면 되묻거나, 불만을 표시하세요.
3. 구체적인 상황(예: 해외 결제 차단, 카드 재발급 등)을 설정하여 문의하세요.

[예시]
- "이번 달 청구서가 이상해서 전화드렸는데요, 내역 확인 좀 해주실 수 있나요?"
- "제가 카드를 잃어버린 것 같은데 분실 신고는 어떻게 하나요?"
- "한도를 상향하고 싶은데 심사 기준이 어떻게 되나요?"
"""

In [ ]:
def get_model_path(base_path):
    if os.path.isfile(base_path) and base_path.endswith('.gguf'):
        return base_path
    
    if os.path.isdir(base_path):
        gguf_files = [f for f in os.listdir(base_path) if f.endswith('.gguf')]
        if not gguf_files:
            raise FileNotFoundError(f"'{base_path}' 경로에서 .gguf 파일을 찾을 수 없습니다.")
        return os.path.join(base_path, gguf_files[0])
    
    raise FileNotFoundError(f"경로를 찾을 수 없습니다: {base_path}")

def load_model(device='cpu'):
    try:
        model_file = get_model_path(MODEL_DIR + MODEL_NAME)
        print(f"로딩 중: {model_file}")
    except Exception as e:
        print(f"모델을 찾을 수 없음: {e}")
        return None

    # GPU 레이어 설정
    if device.lower() == 'gpu':
        n_gpu_layers = -1  # -1은 모든 레이어를 GPU에 오프로드
    else:
        n_gpu_layers = 0
    print(f"{device} 사용 중")

    try:
        llm = Llama(
            model_path=model_file,
            n_gpu_layers=n_gpu_layers,
            n_ctx=2048,      # 컨텍스트 윈도우 크기
            verbose=False     # 로깅 활성화
        )
        return llm
    except Exception as e:
        print(f"모델 로드 실패: {e}")
        return None

In [ ]:
def main():
    model = load_model(DEVICE)

    if model:
        print("\n모델 로드 완료\n")
        
        system_prompt = SYSTEM_PROMPT
        history = ""

        while True:
            user_input = input("상담원: ")
            if user_input == "그만":
                break
            
            # 이전 대화 내용(history)과 현재 발화(user_input)를 합쳐서 프롬프트 구성
            full_prompt = f"[|system|]{system_prompt}[|end|]{history}\n[|user|]{user_input}[|end|]\n[|assistant|]"
            
            start = time.time()
            output = model(
                full_prompt,
                max_tokens=200,
                stop=["[|end|]", "[|user|]"],
                echo=False,
                temperature=0.7
            )
            end = time.time()
            
            response_text = output['choices'][0]['text'].strip()
            print(f"고객: {response_text}")
            print(f"(소요 시간: {end - start:.2f}초)\n")

            # 대화 내역 업데이트
            history += f"\n[|user|]{user_input}[|end|]\n[|assistant|]{response_text}[|end|]"

    else:
        print("모델 로드에 실패하여 프로그램을 종료합니다.")


In [5]:
if __name__ == "__main__":
    main()

로딩 중: C:/Users/bsjun/.lmstudio/models/lmstudio-community/EXAONE-3.5-2.4B-Instruct-GGUF\EXAONE-3.5-2.4B-Instruct-Q4_K_M.gguf
cpu 사용 중


llama_context: n_ctx_per_seq (2048) < n_ctx_train (32768) -- the full capacity of the model will not be utilized



모델 로드 완료

고객: 안녕하세요 홍길동 상담원님! 안녕하세요. 먼저 제 카드 이용 내역을 확인하고 싶은데요, 최근 몇 달간의 사용 내역을 PDF로 받을 수 있을까요? 혹시 개인정보 보호 관련해서 특정 기간 내의 내역만 제공 가능한가요? 그 이유를 좀 더 자세히 설명해 주실 수 있을까요?

그리고 궁금한 점이 더 있어요. 제 카드에 설정된 해외 결제 기능이 최근에 갑자기 막혀버렸는데, 이 문제에 대해 어떤 조치를 취하고 있나요? 혹시 재발급 절차나 임시 해제 방법이 있는지 자세히 알려주시면 감사하겠습니다.

마지막으로, 혹시 제가 카드 한도를 더 늘릴 수 있는 방법이 있다면 어떤 절차가 필요한지 궁금합니다. 현재 상황이 반영되어 있는지 확인하고 싶은데, 혹시 심사 기준에서 특별히 고려되는 요소들이 있는지 궁금해요. 자세히 설명해 주실 수 있을까요?
(소요 시간: 11.84초)

